# **NeuralTennis**

In [1]:
!pip install --quiet torch torchvision opencv-python-headless matplotlib plotly albumentations scikit-learn ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
import os

BASE_DIR = "/content/drive/MyDrive/NeuralTennis"
for sub in ["input", "models", "output"]:
    os.makedirs(f"{BASE_DIR}/{sub}", exist_ok=True)

In [4]:
import cv2

video_path = f"{BASE_DIR}/input/4.mp4"
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise IOError(f"Cannot open {video_path}")
ret, frame = cap.read()
cap.release()
if not ret:
    raise IOError("Cannot read the first frame")
print("Video loaded – resolution:", frame.shape[1], "×", frame.shape[0])

Video loaded – resolution: 1920 × 1080


In [5]:
import cv2

def read_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return frames

def save_video(output_video_frames, output_video_path):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, 30, (output_video_frames[0].shape[1], output_video_frames[0].shape[0]))
    for frame in output_video_frames:
        out.write(frame)
    out.release()
    print(f"Video saved to {output_video_path}")


In [6]:
from ultralytics import YOLO

!yolo task=detect mode=train model=yolov5xu.pt data=/content/drive/MyDrive/NeuralTennis/data/data.yaml epochs=100 imgsz=640 batch=8 patience=15  project=/content/drive/MyDrive/NeuralTennis/models/ball_training name=run

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
100% 186M/186M [00:01<00:00, 122MB/s]
Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/NeuralTennis/data/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v

In [ ]:
# 2. Import e impostazioni
import cv2, os, glob, math

# Cartelle sul tuo Drive
input_dir   = '/content/drive/MyDrive/NeuralTennis/input'
output_dir  = '/content/drive/MyDrive/NeuralTennis/images_to_annotate'
os.makedirs(output_dir, exist_ok=True)

# Parametri
total_target     = 50
video_paths      = sorted(glob.glob(os.path.join(input_dir, '*.mp4')))
num_videos       = len(video_paths)
per_video_target = math.ceil(total_target / num_videos)

global_saved = 0

# 3. Loop su tutti i video, fermandosi a 50 frame totali
for video_path in video_paths:
    if global_saved >= total_target:
        break

    cap       = cv2.VideoCapture(video_path)
    total     = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    step      = max(1, total // per_video_target)
    basename  = os.path.splitext(os.path.basename(video_path))[0]

    saved_in_vid = 0
    frame_idx    = 0
    while cap.isOpened() and saved_in_vid < per_video_target and global_saved < total_target:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_idx % step == 0:
            fname = f'{basename}_frame_{saved_in_vid:02d}.jpg'
            cv2.imwrite(os.path.join(output_dir, fname), frame)
            saved_in_vid += 1
            global_saved += 1
        frame_idx += 1

    cap.release()
    print(f'Video {basename}: estratti {saved_in_vid} frame')

print(f'Totale immagini estratte in {output_dir}: {global_saved}')


Mounted at /content/drive
Video 1: estratti 9 frame
Video 2: estratti 9 frame
Video 3: estratti 9 frame
Video 4: estratti 9 frame
Video 5: estratti 9 frame
Video 6: estratti 5 frame
Totale immagini estratte in /content/drive/MyDrive/NeuralTennis/images_to_annotate: 50


In [ ]:
import json, os, glob
from pathlib import Path

# Path to your images and your manually annotated JSONs
img_dir    = '/content/drive/MyDrive/NeuralTennis/images_to_annotate'
ann_dir    = img_dir  # immagini e JSON vivono nella stessa cartella

# Loop sui 6 match (assumiamo i nomi 1_, 2_, …, 6_)
for match_id in range(1, 7):
    prefix     = f"{match_id}_frame_"
    master_j   = Path(ann_dir) / f"{prefix}00.json"
    if not master_j.exists():
        print(f"[WARN] Master JSON not found for match {match_id} → {master_j}")
        continue

    # Carica il master JSON
    with open(master_j, 'r') as f:
        master_ann = json.load(f)

    # Trova tutti i JPG di questo match
    pattern = str(Path(img_dir) / f"{prefix}*.jpg")
    for img_path in sorted(glob.glob(pattern)):
        target_j = img_path.replace('.jpg', '.json')
        # Se c’è già un JSON (p.es. perché lo hai corretto o perché hai rimosso i frame),
        # lo salti
        if os.path.exists(target_j):
            continue

        # Duplichi il master, aggiornando solo imagePath
        new_ann = master_ann.copy()
        new_ann['imagePath'] = os.path.basename(img_path)
        # (height/width/liquid non cambiano perché tutti i frame hanno stesse dim.)
        with open(target_j, 'w') as out:
            json.dump(new_ann, out, indent=2)
        print(f"Created {os.path.basename(target_j)}")

print("Done.")


Created 1_frame_03.json
Created 1_frame_05.json
Created 2_frame_01.json
Created 2_frame_04.json
Created 2_frame_05.json
Created 3_frame_01.json
Created 3_frame_02.json
Created 3_frame_03.json
Created 3_frame_04.json
Created 3_frame_05.json
Created 3_frame_06.json
Created 3_frame_07.json
Created 3_frame_08.json
Created 4_frame_04.json
Created 5_frame_07.json
Created 6_frame_01.json
Created 6_frame_02.json
Done.


In [ ]:
import json
from pathlib import Path

def labelme_to_coco(labelme_dir: str, output_json: str):
    """
    Convert all LabelMe JSON files in a directory to a single COCO Keypoints JSON.
    Uses the imageWidth/imageHeight fields (1920x1080) to normalize later.
    """
    coco = {
        "images": [],
        "annotations": [],
        "categories": [{
            "id": 1,
            "name": "tennis_court",
            "keypoints": [f"kp{i}" for i in range(14)],
            "skeleton": []
        }]
    }
    ann_id = 1
    img_id = 1

    for jm_file in sorted(Path(labelme_dir).glob("*.json")):
        data = json.load(open(jm_file))
        filename = data['imagePath']
        height = data['imageHeight']  # dovrebbe essere 1080
        width  = data['imageWidth']   # dovrebbe essere 1920

        coco["images"].append({
            "id": img_id,
            "file_name": filename,
            "height": height,
            "width": width
        })

        kpts = [0] * (14 * 3)
        for shape in data["shapes"]:
            idx = int(shape["label"].replace("kp", ""))
            x, y = shape["points"][0]
            kpts[idx*3]   = x
            kpts[idx*3+1] = y
            kpts[idx*3+2] = 2

        coco["annotations"].append({
            "id": ann_id,
            "image_id": img_id,
            "category_id": 1,
            "keypoints": kpts,
            "num_keypoints": sum(1 for v in kpts[2::3] if v>0),
            "bbox": [0, 0, 0, 0],
            "area": 0
        })

        ann_id += 1
        img_id += 1

    with open(output_json, "w") as f:
        json.dump(coco, f)
    print(f"Saved COCO annotations to {output_json}")

# Usage:
labelme_dir  = "/content/drive/MyDrive/NeuralTennis/images_to_annotate"
output_json = "/content/drive/MyDrive/NeuralTennis/court_keypoints_coco.json"
labelme_to_coco(labelme_dir, output_json)


Saved COCO annotations to /content/drive/MyDrive/NeuralTennis/court_keypoints_coco.json


In [6]:
import json, os
from pathlib import Path

ann_path = Path('/content/drive/MyDrive/NeuralTennis/court_keypoints_coco.json')
img_dir  = Path('/content/drive/MyDrive/NeuralTennis/images_to_annotate')

coco = json.loads(ann_path.read_text())
img_ids = {img['id'] for img in coco['images']}
ann_ids = {ann['id'] for ann in coco['annotations']}

# 1) Counts
assert len(coco['images']) == 48, f"Attese 48 immagini, trovate {len(coco['images'])}"
assert len(coco['annotations']) == 48, f"Attese 48 annotazioni, trovate {len(coco['annotations'])}"

# 2) File existence & per‑image 1 annotazione
for img in coco['images']:
    fp = img_dir / img['file_name']
    assert fp.exists(), f"Missing file {fp}"
    anns = [a for a in coco['annotations'] if a['image_id']==img['id']]
    assert len(anns)==1, f"Image {img['id']} ha {len(anns)} annotazioni"

# 3) Keypoints shape + validity
for ann in coco['annotations']:
    kps = ann['keypoints']
    assert len(kps)==42, f"Annotation {ann['id']} ha {len(kps)} keypoints, attesi 42"
    for i in range(0,42,3):
        x,y,v = kps[i], kps[i+1], kps[i+2]
        w,h = coco['images'][[im['id'] for im in coco['images']].index(ann['image_id'])]['width'], \
              coco['images'][[im['id'] for im in coco['images']].index(ann['image_id'])]['height']
        assert 0 <= x <= w and 0 <= y <= h, f"KP fuori immagine in ann {ann['id']}"
        assert v in (0,1,2), f"Visibility errato ({v}) in ann {ann['id']}"
print("✅ COCO JSON OK!")

✅ COCO JSON OK!


In [74]:
#!pip install ultralytics
from ultralytics import YOLO
import cv2
import pandas as pd
import numpy as np
import math
from torchvision.transforms.functional import to_tensor

class PlayerTracker:
    def __init__(self,model_path):
        self.model = YOLO(model_path)

    def detect_frame(self,frame):
        results = self.model.track(frame, persist=True)[0]
        id_name_dict = results.names

        player_dict = {}
        for box in results.boxes:
            track_id = int(box.id.tolist()[0])
            result = box.xyxy.tolist()[0]
            object_cls_id = box.cls.tolist()[0]
            object_cls_name = id_name_dict[object_cls_id]
            if object_cls_name == "person":
                player_dict[track_id] = result

        return player_dict

    def draw_bboxes(self,video_frames, player_detections):
        output_video_frames = []
        for frame, player_dict in zip(video_frames, player_detections):
            # Draw Bounding Boxes
            for track_id, bbox in player_dict.items():
                x1, y1, x2, y2 = bbox
                cv2.putText(frame, f"Player ID: {track_id}",(int(bbox[0]),int(bbox[1] -10 )),cv2.FONT_HERSHEY_SIMPLEX, 0.9, (197, 197, 197), 2)
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (197, 197, 197), 2)
            output_video_frames.append(frame)

        return output_video_frames


class BallTracker:
    def __init__(self,model_path):
        self.model = YOLO(model_path)

    def interpolate_ball_positions(self, ball_positions):
        ball_positions = [x.get(1,[]) for x in ball_positions]

        df_ball_positions = pd.DataFrame(ball_positions,columns=['x1','y1','x2','y2'])

        df_ball_positions = df_ball_positions.interpolate()
        df_ball_positions = df_ball_positions.bfill()

        ball_positions = [{1:x} for x in df_ball_positions.to_numpy().tolist()]

        return ball_positions


    def detect_frame(self,frame):
        results = self.model.predict(frame,conf=0.25)[0]

        ball_dict = {}
        for box in results.boxes:
            result = box.xyxy.tolist()[0]
            ball_dict[1] = result

        return ball_dict


    def draw_bboxes(self,video_frames, player_detections):
        output_video_frames = []
        for frame, ball_dict in zip(video_frames, player_detections):
            # Draw Bounding Boxes
            for track_id, bbox in ball_dict.items():
                x1, y1, x2, y2 = bbox
                cv2.putText(frame, f"Ball ID: {track_id}",(int(bbox[0]),int(bbox[1] -10 )),cv2.FONT_HERSHEY_SIMPLEX, 0.9, (96, 255, 168), 2)
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (96, 255, 168), 2)
            output_video_frames.append(frame)

        return output_video_frames

In [75]:
def main():

    input_video_path = "/content/drive/MyDrive/NeuralTennis/input/1.mp4"
    video_frames = read_video(input_video_path)

    player_tracker = PlayerTracker("/content/drive/MyDrive/NeuralTennis/models/yolov8x.pt")
    player_detections = [player_tracker.detect_frame(frame) for frame in video_frames]
    video_frames = player_tracker.draw_bboxes(video_frames, player_detections)

    ball_tracker = BallTracker("/content/drive/MyDrive/NeuralTennis/models/ball_training/run/weights/best.pt")
    ball_detections = [ball_tracker.detect_frame(frame) for frame in video_frames]
    ball_detections = ball_tracker.interpolate_ball_positions(ball_detections)
    video_frames = ball_tracker.draw_bboxes(video_frames, ball_detections)

    save_video(video_frames, "/content/drive/MyDrive/NeuralTennis/output/1_out.mp4")

if __name__ == "__main__":
    main()


0: 384x640 8 persons, 2 tennis rackets, 2 chairs, 1 clock, 56.9ms
Speed: 3.0ms preprocess, 56.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 2 tennis rackets, 2 chairs, 1 clock, 40.1ms
Speed: 3.1ms preprocess, 40.1ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 1 tennis racket, 2 chairs, 1 clock, 38.7ms
Speed: 2.9ms preprocess, 38.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 2 chairs, 1 clock, 34.8ms
Speed: 2.5ms preprocess, 34.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 1 tennis racket, 2 chairs, 1 clock, 33.2ms
Speed: 2.9ms preprocess, 33.2ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 1 tennis racket, 2 chairs, 1 clock, 33.2ms
Speed: 3.4ms preprocess, 33.2ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 2 chairs, 1 cl